# Using Ansible to Configure Chameleon Instances
In previous tutorials we have gone over how to create and configure instances and [networks](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb#Create-Reservations) in Chameleon using Jupyter Notebooks. One of the benefits of the notebooks is it allows readers to go over each step in the process at their pace. 

Sometimes we would prefer a more automated approach, something that allows us to create labs without requiring user intervention besides starting the build and configure process... and even that can be automated (but will be the subject for another lab). If done right, our automation could be crafted anywhere we wanted, even without needing to access the Chameleon Cloud. So, if we plan on using popular resources on our reservation, we can then get to them 


### Use case


Let's say we want to create a lab with 3 computers arranged as follows:  
[Lab Diagram](picts/Chameleon_3computer_lab.svg)
 

## Limitations
* For this tutorial, we will stick to building the instances themselves using this Jupyter Notebook. Another way would be to use a heat template, which will be covered in another tutorial.

## Getting Started
In this notebook we will
1. Ensure you have a SSH keypair you want to use.
1. Create network
2. Create nodes. We need 3: 1 server and 2 clients just like we did in a recent blog entry on [how to create /etc/hosts for a set of nodes](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb); we will also use that here. Now, We keep building nodes running the centos operating system. Ansible does not care, so this time we will make the two *clients* a centos and an ubuntu one. Note that we are showing another way to create (client) nodes by using two arrays: one of their names and one for their OS.
1. Install ansible on the server.
1. Pull the ansible playbooks from github.
1. Run the playbook installing the software in the two clients. Just to make it more interesting, we will compile it from source.
1. Let playbook also run the test on its own and create output files.
1. Copy the output files to server and then show their output in this Jupyter notebook.

As before, let's define some variables

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
# export OS_PROJECT_NAME='CH-816532'

export LEASE_NAME="$USER-ansibletest"

## Network Information
# Set the names for the network, subnet, router, and switch. 
export PRIVATE_NETWORK_NAME="$USER-ansiblelab"
export PUBLIC_NETWORK_NAME="public"
export SUBNET_NAME="AnsibleSubnet"
export ROUTER_NAME="AnsibleRouter"
export SUBNET_CIDR="192.168.100.0/24"

# The Nodes
export SERVER_NAME="ansibleserver"
# Server's fixed IP inside our private network 
export SERVER_IP='192.168.100.10'
export CLIENT_NAME=('ansibleclient-centos' 'ansibleclient-ubuntu')
export CLIENT_OS=('CC-CentOS7' 'CC-Ubuntu18.04')
export CLIENT_COUNT=2

export SSHKEY_FILE="$HOME/work/.ssh/${USER}-jupyter"
export SSHKEY_NAME="${USER}-jupyter"

# Where is the our Ansible demo repo?
export DEMOREPO="https://github.com/raubvogel/DemoAnsibLab.git"

# export NODE_TYPE="compute_haswell"
export NODE_TYPE="compute_skylake"

export NODE_FLAVOR="baremetal"
export OS_REGION_NAME='CHI@UC'

## SSH keys

As always, we should see if the SSH key we want to use has already been created

In [ ]:
openstack keypair list

and then create and add it to openstack as needed. If you forgot how to do that, you can look into [how to create a SSH keypair](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2FJupyterOrchestration.ipynb#Create-a-SSH-key-pair) in one of the previous webinars' notes.

In [ ]:
[ -f $SSHKEY_FILE ] && echo "ssh keyfile $SSHKEY_FILE already exists" || ssh-keygen -t rsa -b 4096  -P '' -C $SSHKEY_NAME -f $SSHKEY_FILE
openstack keypair create --public-key $SSHKEY_FILE.pub  $SSHKEY_NAME
openstack keypair list

## Create reservations
### Build network
We begin by creating the [reservation (or lease) and the private network](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb#Create-Reservations). Our lease lasts one day from 2 minutes from now.

In [ ]:
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show ${PUBLIC_NETWORK_NAME} -c id -f value)

echo Creating network ${PRIVATE_NETWORK_NAME}
blazar lease-create \
   --physical-reservation min=$((CLIENT_COUNT +1 )),max=$((CLIENT_COUNT +2 )),resource_properties='["=", "$node_type", "'$NODE_TYPE'"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${PRIVATE_NETWORK_NAME}",resource_properties='["==","$physical_network","physnet1"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

RESERVATION=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION $RESERVATION

NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${LEASE_NAME} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo NODE_RESERVATION $NODE_RESERVATION

#### Verify if lease has been created.
Do not continue until `blazar lease-show` below replies with an `ACTIVE` status:

In [ ]:
blazar lease-show --format value -c status "${LEASE_NAME}"

#### Create Network:

An isolated network requires a subnet, router, and external gateway.

In [ ]:
echo "Creating Subnet... "
openstack subnet create --max-width 80 \
                        --subnet-range ${SUBNET_CIDR} \
                        --dhcp \
                        --network ${PRIVATE_NETWORK_NAME} \
                        ${SUBNET_NAME}
                        
echo "Done."
echo "Creating Router... "
openstack router create --max-width 80 ${ROUTER_NAME}
echo "Done."

echo -n "Linking router to subnet... "
openstack router add subnet ${ROUTER_NAME} ${SUBNET_NAME}
echo "Done"

echo -n "Linking router to external gateway... "
openstack router set --external-gateway public ${ROUTER_NAME}
echo "Done"

echo Network ${PRIVATE_NETWORK_NAME} is ready for nodes!

### Build nodes
Following what we learned, we are creating `$SERVER_NAME`, `$CLIENT_NAME[0]` and `$CLIENT_NAME[1]`. 
#### Server
Creating `$SERVER_NAME` is pretty straight forward:

In [ ]:
echo Creating Server ${SERVER_NAME} with fixed internal IP ${SERVER_IP}
openstack server create --max-width 80 \
                        --flavor "${NODE_FLAVOR}" \
                        --image "CC-CentOS7" \
                        --key-name "${SSHKEY_NAME}" \
                        --hint reservation="${NODE_RESERVATION}" \
                        --nic net-id="${PRIVATE_NETWORK_NAME}",v4-fixed-ip="${SERVER_IP}" \
                        --security-group default  \
                        "${SERVER_NAME}"

##### Don't forget the server floating IP

In [ ]:
# Request a public floating IP (in the 'public' network)
FLOATING_IP=$(openstack floating ip create public --format value -c floating_ip_address)
echo "Public/Floating IP for this lab is $FLOATING_IP"
# Associate floating IP to server $SERVER_NAME
openstack server add floating ip $SERVER_NAME $FLOATING_IP 

#### Clients
The client nodes will be more interesting. In a previous webinar we looped over the list of client names. We will do something like that, but in this case the names are in an array. To add to the challenge, we are also looping over the array containing the operating systems we will install in these nodes:

In [ ]:
for ((i = 0; i< ${#CLIENT_NAME[@]}; ++i))
do
  openstack server create \
     --flavor ${NODE_FLAVOR} \
     --image "${CLIENT_OS[i]}" \
     --nic net-id="${PRIVATE_NETWORK_NAME}" \
     --hint reservation="$NODE_RESERVATION" \
     --key-name="${SSHKEY_NAME}" \
     --security-group default  \
     "${CLIENT_NAME[i]}"
done

**Wait for the nodes to become active!** That means running

In [ ]:
openstack server list

until **all three** nodes report `Status = ACTIVE`

While we are here, let's [create a `/etc/hosts` file](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=examples%2Fconfiguration%2FUsingSSHToConfigureNameResolution.ipynb#Build-a-file-that-will-be-appended-to-all-/etc/hosts-files). As a bonus we will also have the clients' IPs ready to be used.

In [ ]:
TEMP_HOST_FILE='/tmp/hostfile.tmp'
echo > $TEMP_HOST_FILE
# Create a string to append to the /etc/host files
echo ${SERVER_IP} ${SERVER_NAME} >> $TEMP_HOST_FILE

for ((i = 0; i < ${#CLIENT_NAME[@]}; ++i ))
do
   NODE_NAME="${CLIENT_NAME[i]}"
   NODE_IP=`openstack server show --format value -c addresses ${NODE_NAME} | tr -d ' ' | cut -d \"=\" -f 2 | cut -d "," -f 1`
   echo $NODE_IP $NODE_NAME >> $TEMP_HOST_FILE
done
cat $TEMP_HOST_FILE

### Ansible
#### Install Ansible
We will also install git since we will need it.

In [ ]:
lease_list_floating_ips $RESERVATION

In [ ]:
login_command="ssh -A -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking no\" -i $SSHKEY_FILE cc@$FLOATING_IP"
eval "$login_command" "sudo yum install -q -y ansible git" 

Ansible uses ssh to connect to the hosts it will be manipulating (let's call them Ansible clients). We do need to create a ssh keypair in `${SERVER_NAME}` 

In [ ]:
eval "$login_command" /bin/bash << EOF
[ -d ~/.ssh ] || mkdir -m 0700 ~/.ssh
ssh-keygen -q -t rsa -N '' -b 4096  -P '' -C "ansible key" -f ~/.ssh/ansiblekey 
EOF

We save the server public key so we can put it later in the clients.

In [ ]:
SERVER_PUBLIC_KEY="$(eval $login_command 'cat ~cc/.ssh/ansiblekey.pub')"
echo $SERVER_PUBLIC_KEY

and then upload the public keys to the clients. While we are there, we might as well upload the `/etc/hosts` file we created earlier (`$TEMP_HOST_FILE`)

In [ ]:
#Start the ssh forwarding agent and add your private key
ssh-agent
ssh-add $SSHKEY_FILE

We only need to upload the `/etc/hosts` file to `${SERVER_NAME}`

In [ ]:
#Copy the host file to the server
scp -o "UserKnownHostsFile=/dev/null" \
    -o "StrictHostKeyChecking=no" \
    -i $SSHKEY_FILE \
    $TEMP_HOST_FILE cc@${FLOATING_IP}:$TEMP_HOST_FILE

#Append the file to the server /etc/host file
eval "$login_command" "cat $TEMP_HOST_FILE | sudo tee -a /etc/hosts"

In [ ]:
eval "$login_command" "sudo bash -c 'cat /tmp/hostfile.tmp >> /moo' "

But for the clients we will upload the private key we saved in `$SERVER_PUBLIC_KEY`. The `/etc/hosts` file will be uploaded with Ansible

In [ ]:
#For each client, copy the the public key
for ((i = 0; i<${#CLIENT_NAME[@]};++i))
do
  NODE_NAME="${CLIENT_NAME[i]}"
  NODE_IP=$(fgrep $NODE_NAME $TEMP_HOST_FILE | cut -d ' ' -f 1)
  echo $NODE_IP $NODE_NAME 

#Append create the ~cc/.ssh dir on the client
  ssh -A \
      -o "UserKnownHostsFile=/dev/null" \
      -o "StrictHostKeyChecking=no" \
      -i $SSHKEY_FILE \
      cc@${FLOATING_IP} \
      "ssh -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking=no\"  cc@${NODE_IP} \"mkdir ~/.ssh && chmod 0700 ~/.ssh && touch ~/.ssh/authorized_keys && chmod 0600 ~/.ssh/authorized_keys\"
  
  #Copy the server public key file to the client
  ssh -A \
      -o "UserKnownHostsFile=/dev/null" \
      -o "StrictHostKeyChecking=no" \
      -i $SSHKEY_FILE \
      cc@${FLOATING_IP} \
      "scp -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking=no\" ~/.ssh/ansiblekey.pub cc@${NODE_IP}:~/.ssh/authorized_keys"
done

#### Ansible
We begin by retrieving the ansible playbook called `DemoAnsibLab`, which is found at `$DEMOREPO` repository.

In [ ]:
eval "$login_command" "git clone $DEMOREPO"

In [ ]:
After we download it, 

Copy the `$TEMP_HOST_FILE` file so we can push it to the clients

In [ ]:
cp $TEMP_HOST_FILE DemoAnsibLab/roles/demolab/files/hosts

We do not need to configure the playbook since we are relying on the server's `/etc/hosts`

In [ ]:
eval "$login_command" /bin/bash << EOF
cd DemoAnsibLab
ansible-playbook demolab.yml
EOF

#### Run Ansible
##### Configure the clients
We are setting their `/etc/hosts` file and hostname.

##### Install and build packages

##### Run and collect data
The performance test is run on the clients by the ansible playbook; we do not have to start it as it is just another task. It creates a file with the output, which we can then copy to the server.

## Show data
Let's copy the test results to the server

And then present them

## Clean up

## Conclusion(s)
1. There are a lot of steps to be done to get to the point we can unleash Ansible. In a future webinar we will show how to automate those steps.
1. The Ansible playbook is performing many tasks on its own, but even though we had to set them up, we do not need to step through each of them unless something breaks.